In [ ]:
%pip install pandas 
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests
%pip install docxtpl
%pip install matplotlib
%pip install tabulate
%pip install pyxirr
%pip install openpyxl
%pip install folium
%pip install html2image

In [ ]:
#local models
from typing import Dict
from models.econometrics import Cost, Currency
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic


#init project
from models.components import Tech
from models.consumption import Energetic
from models.inventory import Building, Project
import pandas as pd

In [ ]:
#local env variables
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Address and general info🌎

## Energy Consumptions🔌

In [ ]:
#cspell:disable


project:Project = Project(
    title='Sistema PV MHNV OnGrid-Mixto',
    building = Building(
        geolocation=(-33.047016, -71.621509),
        name='Edificio Carlos Porter MHNV',
        address='Calle Condell 1546',
        city='Valparaíso'),
    technology= [Tech.PHOTOVOLTAIC],
    consumption= {
        'description':'main',
        'energetic':Energetic.ELI,
        'cost_increment':8/100,
        'consumption':[
            (1051,"01-02-2023",97361),
            (1123,"02-03-2023",104031),
            (1157,"03-04-2023",107181),
            (1220,"04-05-2023",137911),
            (1299,"01-06-2023",146841),
            (1549,"04-07-2023",175101),
            (1353,"02-08-2023",152946),
            (1420,"01-09-2023",160519),
            (1283,"03-10-2023",145033),
            (1275,"03-11-2023",144128),
            (1346,"03-12-2023",152155),
            (1438,"03-01-2024",162554),

            ],
        }
    )

# Integration 

## Machine Components 🎡

In [ ]:
#cspell: disable
#repository of reusable components
from libs.repository import repo,panelRepo
from models.components import Specs
#clean previous load
project.components = {}

from IPython.display import clear_output

from models.energy_storage import Battery
from models.photovoltaic import PvInput
clear_output(wait=True)

# assembly projects components
project.add_generator(
    panelRepo['CS 655W'],
    PvInput(
        description='Panel 655W MODULO 01-PV',
        quantity=9,
        orientation=Orientation(35,28),
        ),
    PvInput(
        description='Panel 655W MODULO 02-PV',
        quantity=11,
        orientation=Orientation(20,28),
        ),
    PvInput(
        description='Panel 655W MODULO 03-PV',
        quantity=4,
        orientation=Orientation(10,28),
    )
    
)

#gen number of panels
pv_quantity = sum(list(map(lambda it:it.quantity,project.components['generación'])))
#nominal power
pv_power = project.nominal_power


project.add_component(
    'Instalación',
    repo['inverter']['Deye 12kW'],# ./../libs/repository.py
    Component(
        description='Medidor',
        specification='Trifasico',
        cost_per_unit=Cost(306_990,Currency.CLP)),
)

#BESS
project.add_component(
    'almacenamiento',
    Component(
        description='Regulador',
        specification=Specs(
        category='Charge Regulator'
        ),
        cost_per_unit=Cost(320_000,Currency.CLP)),
    Component(
        description='Monitor de carga',
        specification=Specs(
        category='Control y operación',
        model='Monitor'
        ),
        cost_per_unit=Cost(150_000,Currency.CLP)
        ),
    # Battery('almacenamiento',
    #             model='MP GEL12-250',
    #             reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-250ah.html',
    #             cost_per_unit=Cost(305990,Currency.CLP),
    #             volt=12,
    #             charge=250,
    #             demand=project.building.consumption_forecast(['main'])['energy'].to_list(),
    #             hours_autonomy=4,
    #             use_regime='8/5',
                # )
)




project.add_component(
    'obras',
    Component(
            description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=pv_quantity*5,
            ),
    Component(
            description='estructura de montaje',
            specification=Specs(
                category='Montaje',
                montaje='coplanar',
                barras='03u /pv',
                anclas='06u /pv',
                anclas='10u /pv',
                ),
            cost_per_unit=Cost(45_000,Currency.CLP),
            quantity=pv_quantity
            ),
    Component(
            description='Instalación Faenas',
            specification=Specs(),
            cost_per_unit=Cost(500_000,Currency.CLP),
            ),
    Component(
            description=' Manual y capacitación',
            specification=Specs(
                taller='2 Hrs',
                manual='2 copias'
                ),
            cost_per_unit=Cost(250_000,Currency.CLP),
            ),
    Component(
            description='Letro',
            specification=Specs(),
            cost_per_unit=Cost(450_000,Currency.CLP),
            )
)


# Write Document 📒


In [ ]:
import os
PATH = 'build/'+f'r_{project.building.city.lower()[:3]}_{project.building.name}/'

if not os.path.exists(PATH):
    #create
    os.makedirs(PATH)

In [1]:
from models.components import Specs

data = {'foo':'10','bar':'20'}
print(" ".join([f"{it[0]}: {it[1]}," for it in data.items()]))

specs = Specs(category='Charge Regulator',brand='Solar SOl',model='SUN9000K',serial='IP C 4459IC',volt='220V')
print(f'{specs:full}')
print(f'{specs:agnostic}')
print(specs)
print(None or 'no data')

foo: 10, bar: 20,

            Charge Regulator Solar SOl SUN9000K
            details     : serial:IP C 4459IC/volt:220V
            market link : no data
            tech link   : no data
            
Charge Regulator Generic N/A model serial:IP C 4459IC/volt:220V
Charge Regulator Solar SOl SUN9000K
no data


### Saving results on EXCEL ❎

In [ ]:
from models.plotter import toTable
toTable(project,PATH)

### Generate Context->Doc📄✍️

In [ ]:
from docxtpl import DocxTemplate # type: ignore
#init doc
#assembly proxy object 
memory_report = DocxTemplate("templates/memory_template.docx")
bidding_report = DocxTemplate("templates/bidding_template.docx")

## Write params and tables

In [ ]:
#loading context
#cspell: disable
memory_report.render(project.context(template=memory_report))
bidding_report.render(project.context(template=bidding_report))


## Insert Plots 📊

In [ ]:
bucket_df = project.bucket_list(Currency.CLP)['bucket']
#cspell: disable
bucket_df[['description','details','quantity','cost_after_tax']]\
    .rename(columns={
                        'description':'descripción',
                        'details':'técnico',
                        'quantity':'cantidad',
                        'cost_after_tax':'costo bruto'
                    }).to_markdown(index=False,floatfmt=',.0f')

In [ ]:
#dep
from models.plotter import plotter
plotter(project,PATH)

In [ ]:
#set into doc
plot_list = [
    'plot_consumption_forecast',
    'plot_irradiance',
    'plot_temperature',
    'plot_components',
    'plot_components_irr',
    'plot_components_production',
    'plot_production_performance',
    'plot_performance_frequency',
    'plot_flux',
    'map_location'
]

for plot in plot_list:
    memory_report.replace_pic(plot,PATH+f'{plot}.png')
    print('replaced plot:',plot)

In [ ]:
from datetime import datetime
memory_report.save(PATH+f"reporte_memoria_calculo.docx")
bidding_report.save(PATH+f"reporte_pliegos_técnicos.docx")
print('work',project,'finish at: ',datetime.now())